<a href="https://colab.research.google.com/github/laresamdeola/Machine-Learning/blob/main/Audio_Engineering_Model_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
import pandas as pd

In [2]:
import os

In [3]:
data_filepaths = os.listdir('/content/sample_data/Audio_Data')

In [4]:
data_filepaths

['Hurry up.wav',
 'Test.MP4',
 'Hurry.m4a',
 'Close.MP4',
 'Has.MP4',
 'Anchor.MP4',
 'Round.MP4',
 'Reduce it.m4a',
 'Frame.MP4',
 'Fragile.m4a',
 'Unannounced.wav',
 'Try.MP4',
 'Let.m4a',
 'Determined.m4a',
 'Keep going.m4a',
 'Love .MP4',
 'Give .MP4',
 'The.MP4',
 'Infiltrated.m4a',
 'Instant information.wav',
 'Right.m4a',
 'Busy.m4a',
 'Class.MP4',
 'Manual transmission.wav',
 'No.m4a',
 'Attain.wav',
 'Elite.m4a',
 'Traffic.wav',
 'Drop.m4a',
 'Amidst.m4a',
 'Tinted Glass.wav',
 'Educate me.wav',
 'Remember me.m4a',
 'True.m4a',
 'Instead.m4a',
 'Court.m4a',
 'Anchor.m4a',
 'Escape route.m4a',
 'Freedom.m4a',
 'Figured out.wav',
 'Eye.MP4',
 'Stop.m4a',
 'How.m4a',
 'Receive.m4a',
 'Reduce it.wav',
 'Has.m4a',
 'Enough.m4a',
 'Dark skinned.m4a',
 'Prove.m4a',
 'Apply.MP4',
 'Excuse me.m4a',
 'Add.m4a',
 'Unannounced.m4a',
 'Limit.MP4',
 'Brain.MP4',
 'Paste.wav',
 'Grab.MP4',
 'Moment of truth.m4a',
 'Attract.MP4',
 'Sad.MP4',
 'Inferior.m4a',
 'Stop it.m4a',
 'Tie.m4a',
 'Clas

In [5]:
os.getcwd()

'/content'

In [6]:
data_filepaths_2 = os.getcwd() + '/sample_data/Audio_Data'

In [7]:
data_filepaths_2

'/content/sample_data/Audio_Data'

In [8]:
os.getcwd()

'/content'

In [9]:
# filepaths

Function to extract features from audio files

In [10]:
def extract_features(filepaths, mfcc=True, chroma=False, mel=False):
  features = []
  for audio_file in filepaths:
      if mfcc:
        data, sample_rate = librosa.load(audio_file, sr=None)
        mfcc_features = librosa.feature.mfcc(y=data, sr=sample_rate)
        mfcc_mean = np.mean(mfcc_features.T, axis=0)
        # mfcc_mean[0:20]
        features.append(mfcc_mean)
      if chroma:
        chroma_features = librosa.feature.chroma_stft(y=audio_file, sr=sample_rate)
        chroma_features = np.mean(chroma_features.T,axis=0)
        features = np.hstack((features, chroma_features))
      if mel:
        mel_features = librosa.feature.melspectrogram(y=audio_file, sr=sample_rate)
        mel_features = np.mean(mel_features.T,axis=0)
        features = np.hstack((features, mel_features))
  return np.array(features)

In [11]:
# extract_features(data_file_paths)

Audio Files to ignore because of error

In [12]:
to_ignore = ['Region.MP4', 'Fight.m4a', 'Fear.MP4', 'Hurry.m4a', 'Ration.MP4', 'Decide.m4a', 'Bill.m4a', 'Race.MP4', 'Ready.MP4']

Methods to prepare dataset

In [13]:
def prepare_dataset(filepaths):
    features, labels = [], []
    for file_path in filepaths:
        # Extract features
        if file_path in to_ignore:
          continue
        extracted_features = extract_features([file_path])
        features.append(extracted_features)
        # Assign label based on filename (assuming filename contains label)
        if 'm4a' in file_path:
          label = 'Noise'
        else:
          label = 'Truth'
        # label = file_path.split('/')[-1].split('-')[0]
        labels.append(label)
    return np.array(features), np.array(labels)

Prepare Dataset Visual Sample

In [14]:
audio_frame = {
    'X': [100, 10, 12],
    'y': ['truth', 'truth', 'noise']
}

audio_frame = pd.DataFrame(audio_frame)

In [15]:
audio_frame

,X,y
0,100,truth
1,10,truth
2,12,noise


Code to extract all the audio data

In [16]:
# data_file_paths = ['Achieve.MP4', 'Add.MP4']

all_file_paths = [os.getcwd() + '/sample_data/Audio_Data/' + i for i in data_filepaths]

In [17]:
X, y = prepare_dataset(all_file_paths)

<ipython-input-10-610f6734038c>:5: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sample_rate = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-10-610f6734038c>:5: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sample_rate = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-10-610f6734038c>:5: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sample_rate = librosa.load(audio_file, sr=None)
/usr/local/l

Features/ Independent Variable

In [18]:
X

array([[[-5.10365021e+02,  1.23619568e+02,  1.68125076e+01, ...,
         -2.75660872e+00,  1.53650987e+00, -4.62637329e+00]],

       [[-4.70466339e+02,  8.25779190e+01,  1.93985677e+00, ...,
          1.90616834e+00, -4.88829756e+00, -1.62862086e+00]],

       [[-2.23629684e+02,  1.10517540e+02,  7.20159674e+00, ...,
          1.69946039e+00,  5.10757685e+00, -6.38594246e-03]],

       ...,

       [[-2.23936081e+02,  1.07812706e+02,  7.77574396e+00, ...,
         -4.36696959e+00,  1.28388453e+00, -3.10572267e+00]],

       [[-2.08925766e+02,  1.21110474e+02,  9.43193150e+00, ...,
         -3.00891548e-01, -2.21732283e+00, -4.11306810e+00]],

       [[-5.81884399e+02,  1.35899780e+02,  1.47958651e+01, ...,
         -3.02202606e+00,  3.82822657e+00,  2.02738667e+00]]],
      dtype=float32)

Labels/Dependent/Target Variable

In [19]:
y

array(['Truth', 'Truth', 'Noise', 'Truth', 'Truth', 'Truth', 'Truth',
       'Noise', 'Truth', 'Noise', 'Truth', 'Truth', 'Noise', 'Noise',
       'Noise', 'Truth', 'Truth', 'Truth', 'Noise', 'Truth', 'Noise',
       'Noise', 'Truth', 'Truth', 'Noise', 'Truth', 'Noise', 'Truth',
       'Noise', 'Noise', 'Truth', 'Truth', 'Noise', 'Noise', 'Noise',
       'Noise', 'Noise', 'Noise', 'Noise', 'Truth', 'Truth', 'Noise',
       'Noise', 'Noise', 'Truth', 'Noise', 'Noise', 'Noise', 'Noise',
       'Truth', 'Noise', 'Noise', 'Noise', 'Truth', 'Truth', 'Truth',
       'Truth', 'Noise', 'Truth', 'Truth', 'Noise', 'Noise', 'Noise',
       'Noise', 'Noise', 'Noise', 'Truth', 'Noise', 'Noise', 'Noise',
       'Noise', 'Noise', 'Noise', 'Noise', 'Truth', 'Truth', 'Noise',
       'Truth', 'Noise', 'Truth', 'Noise', 'Noise', 'Noise', 'Truth',
       'Truth', 'Noise', 'Truth', 'Truth', 'Noise', 'Truth', 'Truth',
       'Noise', 'Truth', 'Truth', 'Noise', 'Noise', 'Truth', 'Truth',
       'Noise', 'Noi

In [20]:
X.ndim

3

Split the dataset into Train and Test set

In [21]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X_train.ndim

3

Reshape the matrices into 2 dimension

In [23]:
X_train_flat = np.reshape(X_train, (X_train.shape[0], -1))
X_test_flat = np.reshape(X_test, (X_test.shape[0], -1))

In [24]:
X_train_flat.ndim

2

Convert string labels to binary labels

In [25]:
y_train_binary = np.where(y_train == "truth", 1, 0)
y_test_binary = np.where(y_test == "truth", 1, 0)

Define and train the deep learning model

In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_flat.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', 'mse'])

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1344      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3457 (13.50 KB)
Trainable params: 3457 (13.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Fit and train the model

In [28]:
model.fit(X_train_flat, y_train_binary, epochs=10, batch_size=32, validation_data=(X_test_flat, y_test_binary))

Epoch 1/10
8/8 [==============================] - 2s 40ms/step - loss: 5.7156 - accuracy: 0.5546 - mse: 0.4226 - val_loss: 7.6992e-07 - val_accuracy: 1.0000 - val_mse: 3.2481e-11
Epoch 2/10
8/8 [==============================] - 0s 6ms/step - loss: 6.3943e-08 - accuracy: 1.0000 - mse: 3.9418e-13 - val_loss: 3.8268e-10 - val_accuracy: 1.0000 - val_mse: 8.6856e-18
Epoch 3/10
8/8 [==============================] - 0s 6ms/step - loss: 1.6528e-11 - accuracy: 1.0000 - mse: 2.1827e-20 - val_loss: 1.3313e-11 - val_accuracy: 1.0000 - val_mse: 1.0567e-20
Epoch 4/10
8/8 [==============================] - 0s 6ms/step - loss: 6.3809e-13 - accuracy: 1.0000 - mse: 3.7658e-23 - val_loss: 3.0980e-12 - val_accuracy: 1.0000 - val_mse: 5.7318e-22
Epoch 5/10
8/8 [==============================] - 0s 10ms/step - loss: 1.2006e-13 - accuracy: 1.0000 - mse: 7.6412e-25 - val_loss: 1.5516e-12 - val_accuracy: 1.0000 - val_mse: 1.4386e-22
Epoch 6/10
8/8 [==============================] - 0s 9ms/step - loss: 6.2928

Using the Decision Tree

In [29]:
# Define and train the Decision Tree classifier
clf = DecisionTreeClassifier()
clf.fit(X_train_flat, y_train)
y_pred = clf.predict(X_test_flat)

# Calculate accuracy for Decision Tree classifier
accuracy = accuracy_score(y_test, y_pred)
print("Decision Tree Accuracy:", accuracy)

Decision Tree Accuracy: 0.9833333333333333


Using the Random Forest

In [30]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_flat, y_train)
rf_predictions = rf_classifier.predict(X_test_flat)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

Random Forest Accuracy: 1.0


Using the Naive Bayes

In [31]:
from sklearn.naive_bayes import GaussianNB

# Naive Bayes Classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train_flat, y_train)
nb_predictions = nb_classifier.predict(X_test_flat)
nb_accuracy = accuracy_score(y_test, nb_predictions)
print("Naive Bayes Accuracy:", nb_accuracy)

Naive Bayes Accuracy: 0.9833333333333333


Using the SVM - Support Vector Machines

In [32]:
from sklearn.svm import SVC

# Support Vector Machine (SVM) Classifier
svm_classifier = SVC()
svm_classifier.fit(X_train_flat, y_train)
svm_predictions = svm_classifier.predict(X_test_flat)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("SVM Accuracy:", svm_accuracy)

SVM Accuracy: 0.9


In [33]:
# Draw the ROC-AUC Curve
# Test with random data.